<a href="https://colab.research.google.com/github/monta0315/pytorch_pra/blob/main/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#パラメータの最適化

モデルを訓練、検証することでデータに対してのモデルのパラメータを最適化し、テストを行う

モデルの訓練は反復的なプロセスになる

各イテレーション（エポック）でモデルは出力を計算し、損失を求める。そして各パラメータについて損失に対する偏微分の値を求める

その後、勾配降下法に基づいてパラメータを最適化する

In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    
    def forword(self,x):
        x = self.flatten(x)
        logits = selft.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


#ハイパーパラメータ

ハイパーパラメータはモデルの最適化プロセスを制御するためのパラメータ

ハイパーパラメータの値が異なると、モデルの学習や収束率に影響する

今回は訓練用のハイパーパラメータとして以下の値を使用する
- Number of Epochs：イテレーション回数
- Batch Size：ミニバッチサイズを構成するデータ数
- Learning Rate：パラメータの更新係数、値が小さいと変化が少なく、大きすぎると訓練に失敗する可能性が生まれる

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#最適化ループ

ハイパーパラメータを設定後、訓練での最適化のループを回すことでモデルを最適化する。

最適化のループの一回のイテレーションはエポックと呼ばれる

各エポックは２種類のループから構成される
- 訓練ループ：データセットに対して訓練を実行し、パラメータを収束させる
- 検証/テストループ：テストデータセットでモデルを評価し、性能が向上しているか確認する

損失関数：Loss Function

データが与えられていても訓練されていないネットワークは正しい答えを出力しない可能性がある

損失関数はモデルが推論した結果と実際の正解との誤差の大きさを測定する関数。訓練ではこの損失関数の値を小さくしている

損失を計算するためには入力データに対するモデルの推論結果を求めその値と正解ラベルの違いを比較する

一般的な損失関数としては、回帰タスクではnn.MSELoss、分類タスクではnn.NLLLossが使用される

nn.CrossEntorpyLossはnn.LogSoftmaxとnn.NLLLossを結合した損失関数となる

モデルが出力するlogitsの値をnn.CrossEntropyLossに与えて正則化し、予測誤差を求める

In [4]:
loss_fn = nn.CrossEntropyLoss()

#最適化器

最適化は各訓練ステップにおいてモデルのごさが小さくなるようにモデルパラメータを調整するプロセス

#最適化アルゴリズム

最適化アルゴリズムは最適化プロセスの具体的な手続き。今回は確率的勾配降下法を使用する

最適化のロジックは全てoptimizerオブジェクトにある

今回はSGDoptimizerを使用する。他にもAdamなど色々ある

訓練したいモデルパラメータをoptimizerに登録し合わせて学習率をハイパーパラメータとして渡すことで初期化している

In [5]:
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

訓練ループ内で最適化は3つのプロセスから構成される

[1]optimizer.zero_grad()を実行し、モデルパラメータの勾配をリセットする

勾配の計算は蓄積されていくので、毎イテレーション、明示的にリセットする

[2]続いて、loss_backwards()を実行しバックプロパゲーションを実行する

PyTorchは損失に対する各パラメータの偏微分の値（勾配）を求めます

[3]最後にoptimizer.step()を実行し、各パラメータの勾配を使用してパラメータの値を調整する


In [6]:
#最適化を実行するコードをループするtrain_loopとテストデータに対してモデルの性能を評価するtest_loopを定義する
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

損失関数とoptimizerを初期化し、それをtrain_loopとtest_loopに渡す


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


NotImplementedError: ignored